In [1]:
# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
from transformers import BertForQuestionAnswering, BertTokenizerFast
import torch

# 指定保存的模型目录
model_save_dir = "/content/drive/MyDrive/Colab_Notebooks/saved_model"

# 加载保存的模型和分词器
model = BertForQuestionAnswering.from_pretrained(model_save_dir)
tokenizer = BertTokenizerFast.from_pretrained(model_save_dir)

# 设置设备
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [3]:
def answer_question(question, context):
    # 对问题和上下文进行分词
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

    # 将分词后的数据移动到设备
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # 使用模型进行预测
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    # 获取答案的开始和结束位置
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1

    # 解码答案
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index]))

    return answer

In [16]:
# 示例问题和上下文
# 示例问题和上下文
# context = "我最近购买了一台高清电视机，但不知道如何进行护理。"
# question = "高清电视机怎么进行护理？"

# context = "我计划去旅行，想购买一个背包。"
# question = "旅行背包有内外袋吗？"

# context = "我正在开发一个软件，想知道如何评估测试用例的有效性。"
# question = "LLM生成测试用例的有效性如何？"

# context = "我在处理一些实例管理问答的任务，需要一些指导。"
# question = "如何对实例管理问答测试？"

# context = "BERT是由Google开发的一种用于自然语言处理的预训练模型。它可以用于各种任务，如问答、文本分类等。"
# question = "BERT是由谁开发的？"

# context = "我出生是2000年1月1号，今年是2023年"
# question = "我生日是？"

context = "我生日是2000年1月1日，今年是2023年"
question = "我生日是？"

# 获取答案
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: 我生日是？
Answer: 2000 年 1 月 1 日


In [19]:
#以下是根据 SQuAD v2.0 数据集的context 和 question 示例：

# context="Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in Santa Clara, California, the home stadium of the San Francisco 49ers."
# question="Who won Super Bowl 50?"

context="The Moon is Earth's only natural satellite. It is the fifth-largest satellite in the Solar System and the largest and most massive relative to its parent planet. The Moon is thought to have formed approximately 4.5 billion years ago, not long after Earth. The most widely accepted explanation is that the Moon formed from the debris left over after a giant impact between Earth and a Mars-sized body called Theia."
question="What is the most widely accepted explanation for the Moon's formation?"
# 获取答案
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")


Question: What is the most widely accepted explanation for the Moon's formation?
Answer: theia
